In [252]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import *
import numpy as np
from urllib.parse import urljoin
import requests
from google.cloud import vision
import os

def calcDelay(n: int) -> float:
    """
    0.2: upper limit (s)
    0.02: lower limit (s)
    """
    return 0.2-0.18/n

opts = Options()
driver = webdriver.Firefox(options=opts)
driver.get("https://play.typeracer.com/")

print(driver.title) 
driver.switch_to.window(driver.current_window_handle)
driver.minimize_window()
driver.maximize_window()
driver.execute_script("window.focus();")

_ = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a.prompt-button.bkgnd-green')))

# You can put the entire code block below into a while loop to endlessly play
# while True:

try:
    button = driver.find_element(By.CSS_SELECTOR, 'a.prompt-button.bkgnd-green')
    button.click()
except:
    button = driver.find_element(By.CSS_SELECTOR, "a.raceAgainLink.raceAgainLink-green")
    button.click()
    
container = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'table.inputPanel.smoothCaretInputBox')))
spans = container.find_elements(By.TAG_NAME, "span")
msg1 = spans[0].text
msg2 = spans[1].text
if len(spans) > 2:
    msg_remainder = spans[2].text
    if msg_remainder == " ":
        msg_remainder=""
    elif msg_remainder[0].isalpha():
        msg_remainder = " "+msg_remainder
else:
    msg_remainder=""
if len(msg2.split()) > 1:
    message = msg1+" "+msg2
else:
    message = msg1+msg2+msg_remainder
print(message)

WebDriverWait(driver, 30).until(
    lambda d: d.find_element(By.CSS_SELECTOR, "input.txtInput").get_attribute("disabled") is None
)

print('Typing!')

text_input = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input.txtInput')))
text_input.clear()
words = message.split()
for i, word in enumerate(words):
    text_input.send_keys(word)
    if i < len(words)-1:
        text_input.send_keys(" ")
    sleep(calcDelay(len(word)))
print('Finished!\n')

# Wait 1 second between each play
# sleep(1)

TypeRacer - Play Typing Games and Race Friends
A sapling must have good soil in order to grow strong. Infertile soil will not bring forth splendid fruit.
Typing!
Finished!



In [253]:
"""
Use this code block to circumvent the verification prompt.

Create a Google Cloud account to use Vision AI API. 
Replace the 'GOOGLE_APPLICATION_CREDENTIALS' with the path to the .json file.
"""

try:
    button = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Begin Test']")))
    button.click()
except:
    button = WebDriverWait(driver, 1).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Begin Re-test']")))
    button.click()

challenge = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'img.challengeImg')))
src = urljoin(driver.current_url, challenge.get_attribute("src"))
session = requests.Session()
for c in driver.get_cookies():
    session.cookies.set(c['name'], c['value'])
resp = session.get(src)
content = resp.content

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "YOUR JSON CREDENTIAL FILE PATH" # .json
client = vision.ImageAnnotatorClient()

vision_image = vision.Image(content=content)
response = client.text_detection(image=vision_image)
if response.error.message:
    raise Exception(response.error.message)
all_text = response.full_text_annotation.text if response.full_text_annotation else ""

ta = driver.find_element(By.CSS_SELECTOR, "textarea.challengeTextArea")
ta.clear()
ta.send_keys(all_text)

button = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[text()='Submit']")))
button.click()

In [251]:
driver.quit() # close Firefox